# Libraries 📚

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import time

# IPNYB
import matplotlib.pyplot as plt

# Autoencoder
# Model and performance
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Reshape, Input, InputLayer

# Code 💻

## Loading saved matrices 

In [ ]:
# Load matrices from previously generated files
X_01 = np.load('matrices/X_01.npy')
y_01 = np.load('matrices/y_01.npy')
X_03 = np.load('matrices/X_03.npy')
y_03 = np.load('matrices/y_03.npy')
nX = np.load('matrices/nX.npy')
ny = np.load('matrices/ny.npy')

## Reviewing Shapes

In [ ]:
print(X_03.shape, y_03.shape)
print(X_01.shape, y_01.shape)
print(nX.shape, ny.shape)

In [ ]:
# Make the 0.1 the standard dataset
X = X_01
y = y_01

## Auto-Encoder Approach

### Train test split

In [ ]:
# Use the matrices to train the model, we will train an autoencoder for the muon events.
# Train test split
        
# Muons --------------------------------------------------------------
print('Muons:')
# Use X and y to recover the muon events using target vector (1 = muon, 0 = antimuon)
muons = X[y == 1]
muon_labels = y[y == 1]

X_train, X_test, y_train, y_test = train_test_split(muons, muon_labels, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

# Normalizing the data
# We will use the function normalize from keras.utils to normalize the data.
# We will normalize the data by dividing the data by the maximum value of the data.
X_train = keras.utils.normalize(X_train, axis=1)
X_val = keras.utils.normalize(X_val, axis=1)
X_test = keras.utils.normalize(X_test, axis=1)

# Reshape the y data to be a column one-hot encoded vector.
y_train = keras.utils.to_categorical(y_train, 2)
y_val = keras.utils.to_categorical(y_val, 2)
y_test = keras.utils.to_categorical(y_test, 2)

# Check the number of records
print('The number of records in the training dataset is', X_train.shape[0])
print('The number of records in the test dataset is', X_test.shape[0])

print(f"Muon shapes: Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}, Train labels: {y_train.shape}, Val labels: {y_val.shape}, Test labels: {y_test.shape}")
# Show target vector (1 = muon, 0 = antimuon)
#print(f"Muon target training vector: {y_train}")

print(' ')
print(' ')

# Antimuons --------------------------------------------------------------
print('Antimuons')
# Use X and y to recover the antimuon events
antimuons = X[y == 0]
antimuon_labels = y[y == 0]

# Train test split for antimuon data
aX_train, aX_test, ay_train, ay_test = train_test_split(antimuons, antimuon_labels, test_size=0.2)
aX_train, aX_val, ay_train, ay_val = train_test_split(aX_train, ay_train, test_size=0.2)

# Normalizing the data
# We will use the function normalize from keras.utils to normalize the data.
# We will normalize the data by dividing the data by the maximum value of the data.
aX_train = keras.utils.normalize(aX_train, axis=1)
aX_val = keras.utils.normalize(aX_val, axis=1)
aX_test = keras.utils.normalize(aX_test, axis=1)

ay_train = keras.utils.to_categorical(ay_train, 2)
ay_val = keras.utils.to_categorical(ay_val, 2)
ay_test = keras.utils.to_categorical(ay_test, 2)

print(' ')
print(' ')

# Check the number of records
print('The number of records in the training dataset is', aX_train.shape[0])
print('The number of records in the test dataset is', aX_test.shape[0])
print(f"Antimuon shapes: Train: {aX_train.shape}, Val: {aX_val.shape}, Test: {aX_test.shape}, Train labels: {ay_train.shape}, Val labels: {ay_val.shape}, Test labels: {ay_test.shape}")

# Neutrons --------------------------------------------------------------
print('Neutrons')
# Normalizing the data
nX = keras.utils.normalize(nX, axis=1)
print(f"Neutron shapes: {nX.shape, ny.shape}")


### Plot events

In [ ]:
# Plotting a muon event
# We will plot a muon event from the training set.
# We will use the function imshow from matplotlib.pyplot to plot the event.
# We will use the parameter cmap to plot the two channels of the event with different colors.
# Plotting the two channels of the event with different colors with shared axis.
# Plot 3 random events from the training set.

random_events = np.random.randint(0, X_train.shape[0], 1)
print(random_events)

for i in random_events:
        fig, axs = plt.subplots(1, 2, figsize=(15, 20))
        axs[0].imshow(X_train[i,:,:,0], cmap='Reds')
        axs[0].set_title('Muon' if y_train[i][0] == 0 else 'Antimuon')
        axs[1].imshow(X_train[i,:,:,1], cmap='Blues')
        axs[1].set_title('Muon' if y_train[i][0] == 0 else 'Antimuon')
        # Show tick labels on both sides
        axs[0].tick_params('x', labelbottom=True, labeltop=True)
        axs[0].tick_params('y', labelleft=True, labelright=True)
        axs[1].tick_params('x', labelbottom=True, labeltop=True)
        axs[1].tick_params('y', labelleft=True, labelright=True)
        axs[0].set_xlabel('Side 1')
        axs[1].set_xlabel('Side 2')
        
plt.show()

### Autoencoder 1

In [ ]:
# Step 5: Autoencoder Model Training
 # We will train the autoencoder model using only the muon training set.
MATRIX_SHAPE = X_train.shape[1:]

# Input layer: the input are images with two channels (side 1 and side 2)
input = tf.keras.layers.Input(shape=(MATRIX_SHAPE))
activation_function = 'relu'

# Encoder layers
encoder = tf.keras.Sequential([
layers.Flatten(),
layers.Dense(np.prod(MATRIX_SHAPE), activation=activation_function),
layers.Dense(X_train.shape[1]*X_train.shape[2], activation=activation_function),
layers.Dense(X_train.shape[1], activation=activation_function)])(input)
#layers.Dense(X_train.shape[2], activation=activation_function)])(input)

# Decoder layers
decoder = tf.keras.Sequential([
#layers.Dense(X_train.shape[2], activation=activation_function),
layers.Dense(X_train.shape[1], activation=activation_function),
layers.Dense(X_train.shape[1]*X_train.shape[2], activation=activation_function),
layers.Dense(np.prod(MATRIX_SHAPE), activation=activation_function),
layers.Reshape(MATRIX_SHAPE)])(encoder)

# Create the autoencoder
autoencoder = Model(input, decoder)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Print the summary of the autoencoder
print(autoencoder.summary())

# Fit the autoencoder
start = time.time()
history = autoencoder.fit(X_train, X_train, 
        epochs=30, 
        validation_data=(X_val, X_val),
        shuffle=True)
end = time.time()
print('Training time:', end - start)


In [ ]:
# Plotting the training and validation loss and accuracy
fig, axs = plt.subplots(1, 2, figsize=(20, 7))
axs[0].plot(history.history['loss'], label='Training loss')
axs[0].plot(history.history['val_loss'], label='Validation loss')
axs[0].set_title('Training and validation loss')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()
axs[0].grid(True)
axs[1].plot(history.history['accuracy'], label='Training accuracy')
axs[1].plot(history.history['val_accuracy'], label='Validation accuracy')
axs[1].set_title('Training and validation accuracy')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy')
axs[1].legend()
axs[1].grid(True)
plt.show()

### Just making sure...

In [ ]:
# Plotting the test loss and accuracy
autoencoder_test_loss, autoencoder_test_accuracy = autoencoder.evaluate(X_test, X_test)
print('Autoencoder test loss:', autoencoder_test_loss)
print('Autoencoder test accuracy:', autoencoder_test_accuracy)


In [ ]:
print(autoencoder.metrics_names)
print(autoencoder_test_loss, autoencoder_test_accuracy)

#### Predicting

In [ ]:
# Plot the difference between the input and the reconstructed output, we want to make sure that data is not separable so we feed the antimuon data to the autoencoder.
# We expect the reconstructed output caused by antimuon data to be non differentiable from the input.

# Predict on the test set
X_test_pred = autoencoder.predict(X_test)
aX_test_pred = autoencoder.predict(aX_test)

# Calculate the mean squared error
X_test_pred_mse_loss = tf.keras.losses.mse(X_test_pred, X_test)
aX_test_pred_mse_loss = tf.keras.losses.mse(aX_test_pred, aX_test)

# Mean over the side and channel axes
X_test_pred_mse_loss = np.mean(X_test_pred_mse_loss, axis=(1,2))
aX_test_pred_mse_loss = np.mean(aX_test_pred_mse_loss, axis=(1,2))

In [ ]:
# Plot the distribution of the reconstruction loss
plt.figure(figsize=(10, 7))
plt.scatter(range(len(X_test_pred_mse_loss)), X_test_pred_mse_loss, color='blue', label='Reconstruction loss muons')
plt.scatter(range(len(aX_test_pred_mse_loss)), aX_test_pred_mse_loss, color='orange', label='Reconstruction loss antimuons')
plt.xlabel('Event ID')
plt.ylabel('Mean Squared Error')
plt.title('MSE Error Reconstruction')
# Plot a horizontal line at 0.05 to set as a threshold
plt.axhline(y=0.005, color='green', linestyle='-', label='Threshold')
plt.legend()
plt.grid(True)
plt.show()


#### Muon vs Reconstructed Muon (same event)

In [ ]:
# Plotting a muon event and its reconstruction
# We will plot a muon event from the training set and its reconstruction from the autoencoder.

random_events = np.random.randint(0, X_test.shape[0], 1)
print(random_events)

for i in random_events:
        fig, axs = plt.subplots(1, 4, figsize=(30, 40))
        
        # Event
        axs[0].imshow(X_test[i,:,:,0], cmap='Reds')
        axs[0].set_title('Muon' if y_test[i][0] == 0 else 'Antimuon')
        axs[1].imshow(X_test[i,:,:,1], cmap='Blues')
        axs[1].set_title('Muon' if y_test[i][0] == 0 else 'Antimuon')
        
        # Reconstructed event
        axs[2].imshow(X_test_pred[i,:,:,0], cmap='Reds')
        axs[2].set_title('Muon' if y_test[i][0] == 0 else 'Antimuon')
        axs[3].imshow(X_test_pred[i,:,:,1], cmap='Blues')
        axs[3].set_title('Muon' if y_test[i][0] == 0 else 'Antimuon')
        
        # Show tick labels on both sides
        axs[0].tick_params('x', labelbottom=True, labeltop=True)
        axs[0].tick_params('y', labelleft=True, labelright=True)
        axs[0].set_xlabel('Side 1')
        
        axs[1].tick_params('x', labelbottom=True, labeltop=True)
        axs[1].tick_params('y', labelleft=True, labelright=True)
        axs[1].set_xlabel('Side 2')
        
        # Show tick labels on both sides
        axs[2].tick_params('x', labelbottom=True, labeltop=True)
        axs[2].tick_params('y', labelleft=True, labelright=True)
        axs[2].set_xlabel('Side 1 - Reconstruction')
        
        axs[3].tick_params('x', labelbottom=True, labeltop=True)
        axs[3].tick_params('y', labelleft=True, labelright=True)
        axs[3].set_xlabel('Side 2 - Reconstruction')
        
plt.show()

#### Antimuon vs Reconstructed Antimuon (same event)

In [ ]:
random_events = np.random.randint(0, aX_test.shape[0], 1)
print(random_events)
for i in random_events:
        fig, axs = plt.subplots(1, 4, figsize=(30, 40))
        
        # Event
        axs[0].imshow(aX_test[i,:,:,0], cmap='Reds')
        axs[0].set_title('Muon' if ay_test[i][0] == 0 else 'Antimuon')
        axs[1].imshow(aX_test[i,:,:,1], cmap='Blues')
        axs[1].set_title('Muon' if ay_test[i][0] == 0 else 'Antimuon')
        
        # Reconstructed event
        axs[2].imshow(aX_test_pred[i,:,:,0], cmap='Reds')
        axs[2].set_title('Muon' if ay_test[i][0] == 0 else 'Antimuon')
        axs[3].imshow(aX_test_pred[i,:,:,1], cmap='Blues')
        axs[3].set_title('Muon' if ay_test[i][0] == 0 else 'Antimuon')
        
        # Show tick labels on both sides
        axs[0].tick_params('x', labelbottom=True, labeltop=True)
        axs[0].tick_params('y', labelleft=True, labelright=True)
        axs[0].set_xlabel('Side 1')
        
        axs[1].tick_params('x', labelbottom=True, labeltop=True)
        axs[1].tick_params('y', labelleft=True, labelright=True)
        axs[1].set_xlabel('Side 2')
        
        # Show tick labels on both sides
        axs[2].tick_params('x', labelbottom=True, labeltop=True)
        axs[2].tick_params('y', labelleft=True, labelright=True)
        axs[2].set_xlabel('Side 1 - Reconstruction')
        
        axs[3].tick_params('x', labelbottom=True, labeltop=True)
        axs[3].tick_params('y', labelleft=True, labelright=True)
        axs[3].set_xlabel('Side 2 - Reconstruction')
        
plt.show()

### Autoencoder 2 (Latent space a little bit bigger)

In [ ]:
def build_autoencoder(img_shape, code_size, activation_function):
    # Encoder
    encoder = Sequential()
    encoder.add(InputLayer(img_shape))
    encoder.add(Flatten())
    encoder.add(Dense(code_size, activation=activation_function))
    
    # Decoder
    decoder = Sequential()
    decoder.add(InputLayer((code_size,)))
    decoder.add(Dense(np.prod(img_shape), activation=activation_function)) # np.prod(img_shape) is the same as 7*5*2, it's more generic than saying 70
    decoder.add(Reshape(img_shape))

    return encoder, decoder

MATRIX_SHAPE = X_train.shape[1:]
print(Input(MATRIX_SHAPE))
activation_function = 'relu'
encoder, decoder = build_autoencoder(MATRIX_SHAPE, 32, activation_function)

inp = Input(MATRIX_SHAPE)
code = encoder(inp)
reconstruction = decoder(code)

autoencoder_2 = Model(inp, reconstruction)
autoencoder_2.compile(optimizer='adam', loss='mse', metrics="accuracy")
print(autoencoder_2.summary())

# Fit the autoencoder
start = time.time()
history = autoencoder_2.fit(X_train, X_train, 
          epochs=30, 
          validation_data=(X_val, X_val),
          shuffle=True)
end = time.time()
print('Training time:', end - start)


In [ ]:
# Plotting the training and validation loss and accuracy
fig, axs = plt.subplots(1, 2, figsize=(20, 7))
axs[0].plot(history.history['loss'], label='Training loss')
axs[0].plot(history.history['val_loss'], label='Validation loss')
axs[0].set_title('Training and validation loss')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()
axs[0].grid(True)
axs[1].plot(history.history['accuracy'], label='Training accuracy')
axs[1].plot(history.history['val_accuracy'], label='Validation accuracy')
axs[1].set_title('Training and validation accuracy')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy')
axs[1].legend()
axs[1].grid(True)
plt.show()

#### Predicting

In [ ]:
# Plot the difference between the input and the reconstructed output, we want to make sure that data is not separable so we feed the antimuon data to the autoencoder.
# We expect the reconstructed output caused by antimuon data to be non differentiable from the input.

# Predict on the test set
X_test_pred = autoencoder_2.predict(X_test)
aX_test_pred = autoencoder_2.predict(aX_test)

# Calculate the mean squared error
X_test_pred_mse_loss = tf.keras.losses.mse(X_test_pred, X_test)
aX_test_pred_mse_loss = tf.keras.losses.mse(aX_test_pred, aX_test)

# Mean over the side and channel axes
X_test_pred_mse_loss = np.mean(X_test_pred_mse_loss, axis=(1,2))
aX_test_pred_mse_loss = np.mean(aX_test_pred_mse_loss, axis=(1,2))

# Plot the distribution of the reconstruction loss
plt.figure(figsize=(10, 7))
plt.scatter(range(len(X_test_pred_mse_loss)), X_test_pred_mse_loss, color='blue', label='Reconstruction loss muons')
plt.scatter(range(len(aX_test_pred_mse_loss)), aX_test_pred_mse_loss, color='orange', label='Reconstruction loss antimuons')
plt.xlabel('Event ID')
plt.ylabel('Mean Squared Error')
plt.title('MSE Error Reconstruction')
# Plot a horizontal line at 0.05 to set as a threshold
plt.axhline(y=0.005, color='green', linestyle='-', label='Threshold')
plt.legend()
plt.grid(True)
plt.show()


#### Muon vs Reconstructed Muon (same event)

In [ ]:
# Plotting a muon event and its reconstruction
# We will plot a muon event from the training set and its reconstruction from the autoencoder.

random_events = np.random.randint(0, X_test.shape[0], 1)
print(random_events)

for i in random_events:
        fig, axs = plt.subplots(1, 4, figsize=(30, 40))
        
        # Event
        axs[0].imshow(X_test[i,:,:,0], cmap='Reds')
        axs[0].set_title('Muon' if y_test[i][0] == 0 else 'Antimuon')
        axs[1].imshow(X_test[i,:,:,1], cmap='Blues')
        axs[1].set_title('Muon' if y_test[i][0] == 0 else 'Antimuon')
        
        # Reconstructed event
        axs[2].imshow(X_test_pred[i,:,:,0], cmap='Reds')
        axs[2].set_title('Muon' if y_test[i][0] == 0 else 'Antimuon')
        axs[3].imshow(X_test_pred[i,:,:,1], cmap='Blues')
        axs[3].set_title('Muon' if y_test[i][0] == 0 else 'Antimuon')
        
        # Show tick labels on both sides
        axs[0].tick_params('x', labelbottom=True, labeltop=True)
        axs[0].tick_params('y', labelleft=True, labelright=True)
        axs[0].set_xlabel('Side 1')
        
        axs[1].tick_params('x', labelbottom=True, labeltop=True)
        axs[1].tick_params('y', labelleft=True, labelright=True)
        axs[1].set_xlabel('Side 2')
        
        # Show tick labels on both sides
        axs[2].tick_params('x', labelbottom=True, labeltop=True)
        axs[2].tick_params('y', labelleft=True, labelright=True)
        axs[2].set_xlabel('Side 1 - Reconstruction')
        
        axs[3].tick_params('x', labelbottom=True, labeltop=True)
        axs[3].tick_params('y', labelleft=True, labelright=True)
        axs[3].set_xlabel('Side 2 - Reconstruction')
        
plt.show()

#### Antimuon vs Reconstructed Antimuon (same event)

In [ ]:
random_events = np.random.randint(0, aX_test.shape[0], 1)
print(random_events)
for i in random_events:
        fig, axs = plt.subplots(1, 4, figsize=(30, 40))        
        
        # Event
        axs[0].imshow(aX_test[i,:,:,0], cmap='Reds')
        axs[0].set_title('Muon' if ay_test[i][0] == 0 else 'Antimuon')
        axs[1].imshow(aX_test[i,:,:,1], cmap='Blues')
        axs[1].set_title('Muon' if ay_test[i][0] == 0 else 'Antimuon')
        
        # Reconstructed event
        axs[2].imshow(aX_test_pred[i,:,:,0], cmap='Reds')
        axs[2].set_title('Muon' if ay_test[i][0] == 0 else 'Antimuon')
        axs[3].imshow(aX_test_pred[i,:,:,1], cmap='Blues')
        axs[3].set_title('Muon' if ay_test[i][0] == 0 else 'Antimuon')
        
        # Show tick labels on both sides
        axs[0].tick_params('x', labelbottom=True, labeltop=True)
        axs[0].tick_params('y', labelleft=True, labelright=True)
        axs[0].set_xlabel('Side 1')
        
        axs[1].tick_params('x', labelbottom=True, labeltop=True)
        axs[1].tick_params('y', labelleft=True, labelright=True)
        axs[1].set_xlabel('Side 2')
        
        # Show tick labels on both sides
        axs[2].tick_params('x', labelbottom=True, labeltop=True)
        axs[2].tick_params('y', labelleft=True, labelright=True)
        axs[2].set_xlabel('Side 1 - Reconstruction')
        
        axs[3].tick_params('x', labelbottom=True, labeltop=True)
        axs[3].tick_params('y', labelleft=True, labelright=True)
        axs[3].set_xlabel('Side 2 - Reconstruction')
        
plt.show()